In [3]:
import requests
from bs4 import BeautifulSoup
import re

url = "https://chemineers.com/events.html"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')
events=[]

event_cards = soup.find_all("div", class_="card")

for card in event_cards:
    title = card.find("h5", class_="card-title")
    desc = card.find("p", class_="card-text")
    clean_title = title.text.strip()
    clean_desc = re.sub(r'\s+', ' ', desc.text.strip())
    if title and desc:
        events.append(f"{clean_title}: {clean_desc}")
with open("chemineers_events_dataset.txt", "w", encoding="utf-8") as f:
    for event in events:
        f.write(event + "\n\n---\n\n")  # separator between entries





In [4]:
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline
import torch

#Loading dataset
with open("chemineers_events_dataset.txt", "r", encoding="utf-8") as f:
    corpus = f.read().split("\n\n---\n\n")

semantic_model = SentenceTransformer("all-MiniLM-L6-v2")  # for semantic search
qa_model = pipeline("question-answering", model="deepset/roberta-base-squad2")  # for answer extraction

# for embeddings
corpus_embeddings = semantic_model.encode(corpus, convert_to_tensor=True)

def get_response(query):
    query_embedding = semantic_model.encode(query, convert_to_tensor=True)
    scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    best_idx = torch.argmax(scores).item()
    best_context = corpus[best_idx]

    qa_result = qa_model({
        "question": query,
        "context": best_context
    })

    if qa_result["score"] > 0.3 and qa_result["answer"].strip() != "":
        return f" Answer: {qa_result['answer']}"
    else:
        return f" Info:\n{best_context}"


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Device set to use cpu


In [6]:

query ="Exergy was inaugrated in which year?"
result = get_response(query)
print(result)


/usr/local/lib/python3.11/dist-packages/transformers/pipelines/question_answering.py:390: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


 Answer: 2023
